In [ ]:
pip uninstall -y qfinuwa

In [ ]:
!pip install qfinuwa

In [1]:
import pandas as pd
from qfinuwa import Indicators, Strategy, Backtester, Plotting

In [52]:
class CustomIndicators(Indicators):
    
    @Indicators.MultiIndicator
    def bollinger_bands(self, stock, BOLLINGER_WIDTH = 2, WINDOW_SIZE = 50):

        mid_price = (stock['high'] + stock['low']) / 2
        rolling_mid = mid_price.rolling(WINDOW_SIZE).mean()
        rolling_std = mid_price.rolling(WINDOW_SIZE).std()

        return {"upper_bollinger": rolling_mid + BOLLINGER_WIDTH*rolling_std,
                "lower_bollinger": rolling_mid - BOLLINGER_WIDTH*rolling_std}

    @Indicators.SingleIndicator
    def pairs_ratio(self, stocks, WINDOW_SIZE = 30):
        vgt_mid_price = (stocks['VGT']['high'] + stocks['VGT']['low']) / 2
        iyw_mid_price = (stocks['IYW']['high'] + stocks['IYW']['low']) / 2
    
        price_ratio = vgt_mid_price / iyw_mid_price

        rolling_mean = price_ratio.rolling(window=WINDOW_SIZE).mean()
        rolling_std = price_ratio.rolling(window=WINDOW_SIZE).std()
        z_score = (price_ratio - rolling_mean) / rolling_std
        return {"z_score" : z_score}

In [36]:
ci = CustomIndicators()
data = ['VGT', 'IYW']
stocks = {d: pd.read_csv('data/' + d + '.csv') for d in data}
z_score = ci.pairs_ratio(stocks)
z_score.describe()

count    431143.000000
mean         -0.026137
std           1.464486
min          -5.561893
25%          -1.646939
50%          -0.049239
75%           1.647012
max           5.294654
dtype: float64

In [12]:
class MyStrategy(Strategy):
    
    def __init__(self, quantity=5):
        self.quantity = quantity
        self.n_buys = 0
        self.n_sells = 0
    
    def on_data(self, prices, indicators, portfolio):

        for stock in portfolio.stocks:
            # Buy if current price is below lower bollinger band
            if(prices['close'][stock][-1] < indicators['lower_bollinger'][stock][-1]):
                portfolio.order(stock, quantity = self.quantity)
                self.n_buys += 1
            
            # Sell if current price is above upper bollinger band
            if(prices['close'][stock][-1] > indicators['upper_bollinger'][stock][-1]):
                portfolio.order(stock, quantity = -self.quantity)
                self.n_sells += 1

    def on_finish(self):
        # Added to results object - access using result.on_finish
        return self.n_buys, self.n_sells

In [63]:
class PairsTrading(Strategy):

    def __init__(self, quantity=5):
        self.quantity = quantity
        self.vgt_position = 0
        self.iyw_position = 0

    def on_data(self, prices, indicators, portfolio): 
        # On average, VGT is 4.05 times more expensive than IYW

        # If z_score is greater than 3, buy VGT and sell IYW
        if indicators['z_score'][-1] > 3:
            portfolio.order('VGT', quantity = self.quantity)
            portfolio.order('IYW', quantity = -4*self.quantity)
            self.vgt_position += self.quantity
            self.iyw_position -= self.quantity
        
        # If z_score is greater than 1.5, buy VGT and sell IYW to half delta limit
        if indicators['z_score'][-1] > 1.5 and abs(self.vgt_position < 500) and abs(self.iyw_position < 500):
            portfolio.order('VGT', quantity = self.quantity)
            portfolio.order('IYW', quantity = -4*self.quantity)
            self.vgt_position += self.quantity
            self.iyw_position -= self.quantity
        
        # If z_score is less than -3, buy IYW and sell VGT
        elif indicators['z_score'][-1] < -1.5 and abs(self.vgt_position < 500) and abs(self.iyw_position < 500):
            portfolio.order('VGT', quantity = -self.quantity)
            portfolio.order('IYW', quantity = 4*self.quantity)
            self.vgt_position -= self.quantity
            self.iyw_position += self.quantity

        # If z_score is less than -1.5, buy IYW and sell VGT to half delta limit
        elif indicators['z_score'][-1] < -1.5 and abs(self.vgt_position < 500) and abs(self.iyw_position < 500):
            portfolio.order('VGT', quantity = -self.quantity)
            portfolio.order('IYW', quantity = 4*self.quantity)
            self.vgt_position -= self.quantity
            self.iyw_position += self.quantity
        
        # If z_score is between -0.5 and 0.5, exit positions
        elif abs(indicators['z_score'][-1]) < 0.5:
            portfolio.order('VGT', quantity = -self.vgt_position)
            portfolio.order('IYW', quantity = -self.iyw_position)
        return
    
    def on_finish(self):
        return self.vgt_position, self.iyw_position
        

In [64]:
backtester = Backtester(PairsTrading, CustomIndicators, ['VGT', 'IYW'], data_folder='data', days=30, fee=0.000, delta_limits=1000)
backtester

> Precompiling data: 100%|██████████| 431430/431430 [00:04<00:00, 89077.77it/s]


Backtester:
- Strategy: PairsTrading
	- Params: {'quantity': 5}
- Indicators: CustomIndicators
	- Params: {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 50}, 'pairs_ratio': {'WINDOW_SIZE': 30}}
	- SingleIndicators: ['z_score']
	- MultiIndicators: ['lower_bollinger', 'upper_bollinger']
- Stocks: ['IYW', 'VGT']
- Fee 0.0
- delta_limit: {'IYW': 1000, 'VGT': 1000}
- Days: 30

In [66]:
backtester.run(cv=10)

> Running backtest over 10 samples of 30 days: 100%|██████████| 10/10 [00:08<00:00,  1.11it/s]



{'strategy': {'quantity': 5}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 50}, 'pairs_ratio': {'WINDOW_SIZE': 30}}}

Mean ROI:	114386.53502349588
STD ROI:	149288.2805053217

14/03/2022 -> 12/04/2022:	143971.360
14/01/2022 -> 11/02/2022:	-28598.158
05/07/2021 -> 03/08/2021:	22874.709
10/05/2022 -> 08/06/2022:	262277.281
19/10/2021 -> 17/11/2021:	275591.675
10/10/2022 -> 08/11/2022:	-20628.735
04/08/2021 -> 02/09/2021:	394127.053
12/10/2021 -> 10/11/2021:	152581.824
15/07/2022 -> 12/08/2022:	26329.733
20/10/2022 -> 18/11/2022:	-84661.392

AVERAGED RESULTS FOR 10 RUNS:
|               |     IYW |      VGT |     Net |
|---------------|---------|----------|---------|
| n_trades      |   278.2 |    362.1 |   640.3 |
| n_buys        |      80 |    258.6 |   338.6 |
| n_sells       |   198.2 |    103.5 |   301.7 |
| gross_pnl     |  125213 | -10826.1 |  114387 |
| fees_paid     |       0 |        0 |       0 |
| net_pnl       |  125213 | -10826.1 |  114387 |
| pnl_p

In [69]:
# backtester.run_grid_search(
#     strategy_params={'quantity': [1, 5, 10]},
#     indicator_params={'bollinger_bands': {'BOLLINGER_WIDTH': [1, 2], 'WINDOW_SIZE': [10, 50, 100]}},
#     cv=10)

backtester.run_grid_search(
    strategy_params={'quantity': [5, 10]},
    indicator_params={'pairs_ratio': {'WINDOW_SIZE': [10, 30, 50, 100]}},
    cv=20)

> Backtesting the across the following ranges:
Agorithm Parameters {'quantity': [5, 10]}
Indicator Parameters {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 50}, 'pairs_ratio': {'WINDOW_SIZE': [10, 30, 50, 100]}}


Running paramter sweep (cv=20): 100%|██████████| 8/8 [02:00<00:00, 15.05s/it]


Best parameter results:

{'strategy': {'quantity': 5}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 50}, 'pairs_ratio': {'WINDOW_SIZE': 30}}}

Mean ROI:	156909.20364754897
STD ROI:	93090.73333407863

24/05/2022 -> 22/06/2022:	27583.434
22/07/2022 -> 19/08/2022:	-12033.717
05/05/2022 -> 03/06/2022:	171005.386
19/07/2022 -> 17/08/2022:	270222.067
04/03/2022 -> 01/04/2022:	138562.573
28/01/2022 -> 25/02/2022:	59917.444
27/07/2021 -> 25/08/2021:	21183.580
29/09/2021 -> 28/10/2021:	316272.082
22/07/2021 -> 20/08/2021:	201960.922
15/06/2021 -> 14/07/2021:	174107.061
26/07/2022 -> 24/08/2022:	238021.482
09/02/2022 -> 10/03/2022:	162223.991
07/02/2022 -> 08/03/2022:	109613.841
03/08/2022 -> 01/09/2022:	241547.968
18/03/2022 -> 15/04/2022:	220591.896
17/09/2021 -> 15/10/2021:	323980.553
09/04/2021 -> 07/05/2021:	108221.926
20/04/2021 -> 19/05/2021:	128155.390
04/01/2022 -> 02/02/2022:	68501.283
07/09/2021 -> 06/10/2021:	168544.909

AVERAGED RESULTS FOR 20 RUNS:
|      